## Create Folder Structure

In [42]:
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd


with open('questions.html', 'r') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    rows = soup.find('tbody').find_all('tr')
    data = [row.find_all('td') for row in rows] # [[td, td, td], [td, td, td], ...]

questions = [{
    'company': td[0].find('p').text.strip()
    , 'title': td[1].find('p').text.strip()
    , 'category': td[2].find('a').text.lower().strip()
    , 'difficulty': td[3].find('a').text.lower().strip()
    , 'url': 'https://datalemur.com/questions' + td[4].find('a')['href']
    , 'file_name': td[4].find('a')['href'].split('/')[-1] + '.sql'
    } for td in data]

In [43]:
''.join(sorted(set(''.join([q['file_name'] for q in questions]))))

'&-.01247abcdefghijklmnopqrstuvwxyz'

In [50]:
for q in [q for q in questions if q['category'] == 'sql']:
    file_path = Path(q['category'], q['difficulty'])
    file_name = file_path.joinpath(q['file_name'])
    
    # create directory / file
    file_path.mkdir(parents=True, exist_ok=True)
    file_name.touch(exist_ok=True)
    
    comment = f"-- {q['url']}"
    blank = '\nSELECT NULL AS blank;\n'
    with open(file_name, 'r+') as f:
        contents = f.readlines() # keep existing contents
        f.seek(0) # go to beginning of file

        if len(contents) == 0:
            f.writelines([comment] + [blank])
        # elif any(c.strip() == blank.strip() for c in contents):
        #     f.truncate(0)
        #     f.writelines([comment] + [blank])
            
        elif contents[0].strip() != comment.strip():
            f.truncate(0)
            f.writelines([comment] + contents)

In [5]:
pd.DataFrame(questions).to_parquet('questions.parquet', index=False)